# Variables

In [1]:
ga_input = '/grid/home/nbourgeois/wip/lau_lyn_cdo/gaa.pep.fas' #sequence pep.fas Input
tissue = 'Muscle_Skeletal' # Tissue type for CoCoPuts
filename = 'bai_1'

# Imports


In [2]:
import os
cwd = os.getcwd()

## Import the code for codon opt
import sys
sys.path.insert(1, '/grid/home/nbourgeois/codonOpt')
# from run_pygad import *
from general_functions import *
from dyn_prog import *

import os
cwd = os.getcwd()
pygad_loc = '/grid/home/nbourgeois/codonOpt/'
os.chdir(pygad_loc)
import pygad
import numpy as np
from general_functions import *
from metrics import *
import time
import glob
import multiprocessing
from general_functions import *
from metrics import *
import shutil
import sys
os.chdir(cwd)

Using TensorFlow backend.


# Data

In [3]:
(keys, seqs) = readFasta(ga_input)
if len(seqs) == 1:
    aa_seq=str(seqs[0])

In [4]:
def predictSites_multi( fastaIds,fastaSeqs,threshold=.1):

    gpu0_avail = gpu_memory_usage(0)
    gpu1_avail = gpu_memory_usage(1)
    if gpu0_avail + gpu1_avail > 30000:
        print('Out of memory\n')
        return(-1)
    if gpu0_avail < gpu1_avail:
        gpu="0"
    else:
        gpu="1"
    
    os.environ["CUDA_VISIBLE_DEVICES"]=gpu

    from keras.models import load_model
    import tensorflow as tf
    from tensorflow import keras

    results = {}
    gpus =tf.config.experimental.list_physical_devices('GPU')

    try: 
        tf.config.experimental.set_virtual_device_configuration(gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5000)])
        
        context = 10000
        keras.backend.set_learning_phase(0)
        paths = ('models/spliceai{}.h5'.format(x) for x in range(1, 6))
        models = [load_model(resource_filename('spliceai', x), compile=False ) for x in paths]
        for seq_id, seq in zip(fastaIds,fastaSeqs):

            x = one_hot_encode('N'*(context//2) + str(seq) + 'N'*(context//2))[None, :]
            y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
            acceptor_prob = y[0, :, 1]
            donor_prob = y[0, :, 2]
            coord = list( range(1, len( donor_prob )+1 ) )
                    
            # custom metric
#             results[seq_id] = (len(acceptor_prob[acceptor_prob > threshold]) + len(donor_prob[donor_prob > threshold]))/2
            acceptor_count = 0
            donor_count = 0
            # for i in coord:
            #     if donor_prob[i] > threshold :
            #         donor_count += donor_prob[i]
            #     if acceptor_prob[i] > threshold and donor_count > 0:
            #         acceptor_count += acceptor_prob[i]
            # custom metric
            results[seq_id] = {'Coord':coord,'Donor':donor_prob,'Acceptor':acceptor_prob}
            # results.append(acceptor_count * donor_count)
        return(results)
    except tf.errors as e:
        # Splice AI custome sequeence prediction
        print(e,'\n')
        return(-1)

# Genetic Algorithm

In [5]:
optimizer = Optimizer(aa_seq,tissues=tissue )
optimized_seq = optimizer.optimize()
get_bai(optimized_seq,optimizer.bai_weight_dict[tissue])

952 951 950 949 948 947 946 945 944 943 942 941 940 939 938 937 936 935 934 933 932 931 930 929 928 927 926 925 924 923 922 921 920 919 918 917 916 915 914 913 912 911 910 909 908 907 906 905 904 903 902 901 900 899 898 897 896 895 894 893 892 891 890 889 888 887 886 885 884 883 882 881 880 879 878 877 876 875 874 873 872 871 870 869 868 867 866 865 864 863 862 861 860 859 858 857 856 855 854 853 852 851 850 849 848 847 846 845 844 843 842 841 840 839 838 837 836 835 834 833 832 831 830 829 828 827 826 825 824 823 822 821 820 819 818 817 816 815 814 813 812 811 810 809 808 807 806 805 804 803 802 801 800 799 798 797 796 795 794 793 792 791 790 789 788 787 786 785 784 783 782 781 780 779 778 777 776 775 774 773 772 771 770 769 768 767 766 765 764 763 762 761 760 759 758 757 756 755 754 753 752 751 750 749 748 747 746 745 744 743 742 741 740 739 738 737 736 735 734 733 732 731 730 729 728 727 726 725 724 723 722 721 720 719 718 717 716 715 714 713 712 711 710 709 708 707 706 705 704 703 

0.9169638283374133

In [6]:
optimized_seq2 = optimizer.trim(307)
get_bai(optimized_seq2,optimizer.bai_weight_dict[tissue])

952 951 950 949 948 947 946 945 944 943 942 941 940 939 938 937 936 935 934 933 932 931 930 929 928 927 926 925 924 923 922 921 920 919 918 917 916 915 914 913 912 911 910 909 908 907 906 905 904 903 902 901 900 899 898 897 896 895 894 893 892 891 890 889 888 887 886 885 884 883 882 881 880 879 878 877 876 875 874 873 872 871 870 869 868 867 866 865 864 863 862 861 860 859 858 857 856 855 854 853 852 851 850 849 848 847 846 845 844 843 842 841 840 839 838 837 836 835 834 833 832 831 830 829 828 827 826 825 824 823 822 821 820 819 818 817 816 815 814 813 812 811 810 809 808 807 806 805 804 803 802 801 800 799 798 797 796 795 794 793 792 791 790 789 788 787 786 785 784 783 782 781 780 779 778 777 776 775 774 773 772 771 770 769 768 767 766 765 764 763 762 761 760 759 758 757 756 755 754 753 752 751 750 749 748 747 746 745 744 743 742 741 740 739 738 737 736 735 734 733 732 731 730 729 728 727 726 725 724 723 722 721 720 719 718 717 716 715 714 713 712 711 710 709 708 707 706 705 704 703 

0.9156710688492328

In [7]:
# optimizer = Optimizer(aa_seq, tissues=tissue, ntissues='Liver')
# optimized_seq2 = optimizer.optimize()
# optimizer.bai_weight_dict.keys()

In [10]:
sps_results3 = predictSites_multi(['seq','seq2'],[optimized_seq,optimized_seq2],.3)


2022-08-11 16:13:10.257205: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-08-11 16:13:10.273039: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla V100-PCIE-16GB major: 7 minor: 0 memoryClockRate(GHz): 1.38
pciBusID: 0000:d7:00.0
2022-08-11 16:13:10.273271: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2022-08-11 16:13:10.275589: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2022-08-11 16:13:10.277225: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0
2022-08-11 16:13:10.277527: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10.0
2022-08-11 16:13:10.279523: I tensorflow/stream_executor/platfo

ERROR! Session/line number was not unique in database. History logging moved to new session 5286


/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusparse.so.10.0
2022-08-11 16:13:10.285502: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2022-08-11 16:13:10.285953: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1746] Adding visible gpu devices: 0
2022-08-11 16:13:10.320210: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2022-08-11 16:13:10.341352: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000000000 Hz
2022-08-11 16:13:10.354962: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x564a45ca1e70 executing computations on platform Host. Devices:
2022-08-11 16:13:10.354986: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
2022-08-11 16:13:10.355355: I tensorflow/core/common

In [15]:
thresh=.01
df_all = pd.DataFrame(sps_results3['seq'])
df_t = df_all.loc[(df_all['Acceptor']>thresh) | (df_all['Donor']>thresh)]
df_t

,Coord,Donor,Acceptor
306,307,0.012696,0.000044
486,487,0.000115,0.029419
1436,1437,0.014637,0.000026
2046,2047,0.000041,0.010747
2145,2146,0.000034,0.014789
2285,2286,0.016797,0.000212
2481,2482,0.000025,0.019808
2786,2787,0.000001,0.019972


In [14]:
thresh=.01
df_all = pd.DataFrame(sps_results3['seq2'])
df_t = df_all.loc[(df_all['Acceptor']>thresh) | (df_all['Donor']>thresh)]
df_t

,Coord,Donor,Acceptor
486,487,0.000128,0.022968
1436,1437,0.014156,0.000025
2046,2047,0.000039,0.010895
2145,2146,0.000035,0.014257
2285,2286,0.015246,0.000204
2481,2482,0.000024,0.018587
2786,2787,0.000001,0.019634


In [13]:
# get_bai(optimized_seq,optimizer.bai_weight_dict['Liver']),get_bai(optimized_seq,optimizer.bai_weight_dict['Heart_Atrial_Appendage'])
# get_bai(optimized_seq2,optimizer.bai_weight_dict['Liver']),get_bai(optimized_seq2,optimizer.bai_weight_dict['Heart_Atrial_Appendage'])
